# 1. 🪙 Introducción a la Arquitectura Medallón

La **arquitectura medallón** es un patrón de diseño ampliamente utilizado en entornos de **ingeniería de datos moderna**, especialmente en **Databricks** y **Delta Lake**.  
Su objetivo es **estructurar los datos en capas sucesivas de calidad**, asegurando trazabilidad, gobernanza y rendimiento en los procesos analíticos.

---

### Estructura clásica

| Capa | Nombre | Propósito principal | Tipo de datos |
|------|---------|--------------------|----------------|
| 🥉 **Bronze** | *Raw / Landing* | Contiene los datos tal y como se reciben desde la fuente. Sin limpiar ni transformar. | Crudos, semi-estructurados o sin esquema definido |
| 🥈 **Silver** | *Clean / Validated* | Aplica reglas de validación, limpieza, tipado y estandarización. Aquí se descartan o marcan los registros inválidos. | Limpios, consistentes, conformes con el modelo lógico |
| 🥇 **Gold** | *Curated / Business-ready* | Contiene datos agregados o listos para análisis, reporting y dashboards. | Métricas y KPIs de negocio |

Este enfoque en capas permite:
- **Controlar la calidad** y trazabilidad de los datos.  
- **Facilitar la depuración** de errores o inconsistencias.  
- **Optimizar el rendimiento** mediante particiones y Z-ORDER.  
- **Habilitar auditorías y lineage completo** desde origen hasta capa de consumo.  

---

# ⚙️ Extensión del modelo: Tabla de Cuarentena

Durante la **validación en la capa Silver**, algunos registros no cumplen las reglas de calidad (por ejemplo, tipos erróneos, valores nulos críticos o claves no existentes).  
En lugar de descartarlos definitivamente, se almacenan en una **tabla de cuarentena** —paralela a la Silver— que guarda **los registros rechazados** junto con información del error detectado.

Esto permite:
- **Preservar trazabilidad total**, sin pérdida de información.  
- **Revisar y corregir manual o automáticamente** los registros problemáticos.  
- **Retroalimentar la capa Silver** una vez corregidos.  

---

### Flujo extendido del modelo

```mermaid
flowchart TD
    A[Bronze 🥉 <br/> (Raw / Landing)] --> B[Silver 🥈 <br/> (Validated)]
    B --> C[Gold 🥇 <br/> (Business-ready)]
    B --> D[[Quarantine 🧰 <br/> (Errores detectados)]]
    D -->|Registros corregidos| B


# 2. Descripcion de los Data Sets

%md
## 1) `fleet_dim_aircraft.csv` (tabla de dimensión)
Catálogo maestro de aeronaves.

- **aircraft_id** *(STRING)*: Identificador interno único del avión (clave para joins).
- **tail_number** *(STRING)*: Matrícula de la aeronave (p. ej., EC-XXX).
- **model** *(STRING)*: Modelo (p. ej., A320-214, A321neo).
- **manufacturer** *(STRING)*: Fabricante (p. ej., Airbus).
- **engine_model** *(STRING)*: Modelo de motor (p. ej., CFM56-5B4).
- **first_service_dt** *(DATE)*: Fecha de entrada en servicio.
- **status** *(STRING)*: Estado operativo (Active | Maintenance | Retired).
- **cycles_total** *(INT)*: Ciclos acumulados (despegue/aterrizaje).
- **hours_total** *(DOUBLE)*: Horas de vuelo acumuladas.

**Relación:** Se une con las tablas de hechos por `aircraft_id`.

---

## 2) `ops_fact_engine_sensor.csv` (tabla de hechos – telemetría simplificada)
Muestras de telemetría (baja resolución para demo) y **12 columnas**.

- **reading_id** *(STRING)*: Identificador único de la lectura (UUID).
- **aircraft_id** *(STRING)*: FK a `fleet_dim_aircraft.aircraft_id`.
- **event_ts** *(TIMESTAMP, UTC en ISO8601)*: Marca temporal del evento.
- **event_date** *(DATE)*: Fecha derivada de `event_ts` (útil para particionado).
- **engine_pos** *(TINYINT)*: Posición del motor (1=izq., 2=dcha.).
- **phase_of_flight** *(STRING)*: Fase del vuelo (Taxi/Takeoff/Climb/Cruise/Descent/Landing).
- **altitude_ft** *(INT)*: Altitud estimada en pies.
- **ias_kts** *(INT)*: Indicated Airspeed (nudos).
- **egt_c** *(DOUBLE)*: Exhaust Gas Temperature (°C).
- **n1_pct** *(DOUBLE)*: % de RPM del fan (N1).
- **fuel_flow_kg_h** *(DOUBLE)*: Caudal de combustible (kg/h).
- **vib_ips** *(DOUBLE)*: Vibración en pulgadas/segundo.

**Uso típico:** generación de *features* de ventana por `aircraft_id` + `event_ts` (p. ej., medias móviles de `egt_c`, `vib_ips`, `n1_pct`).

---

## 3) `mro_fact_maintenance_event_50k.csv` (tabla de hechos – 50k filas)
Histórico de eventos de mantenimiento, inspecciones y correctivos.

- **event_id** *(STRING)*: Identificador del evento (UUID).
- **aircraft_id** *(STRING)*: FK a `fleet_dim_aircraft.aircraft_id`.
- **event_ts** *(TIMESTAMP, UTC en ISO8601)*: Fecha y hora del evento.
- **event_date** *(DATE)*: Fecha derivada (útil para particionado).
- **event_type** *(STRING)*: Tipo de evento (Inspection | A-Check | Unscheduled | Corrective | Replacement).
- **ata_chapter** *(STRING)*: Capítulo ATA (sistema afectado, p. ej., “72 Engine”).
- **defect_code** *(STRING)*: Código interno de defecto.
- **severity** *(STRING)*: Severidad percibida (Low | Medium | High | Critical).
- **grounded** *(BOOLEAN)*: Si el avión quedó en tierra (AOG) por el evento.
- **workorder_id** *(STRING)*: Orden de trabajo en el sistema MRO/ERP.
- **duration_min** *(INT)*: Duración de la intervención (minutos).
- **description** *(STRING)*: Resumen libre del trabajo/defecto.
- **next_due_cycles** *(INT, nullable)*: Ciclos restantes hasta el siguiente check (si aplica).
- **next_due_hours** *(DOUBLE, nullable)*: Horas restantes hasta el siguiente check (si aplica).

**Uso típico:** sirve como **etiqueta/target** para modelos supervisados (p. ej., probabilidad de evento “Unscheduled/Corrective” en ≤ X horas).

---

## Recomendaciones de modelado
- **Particionado por fecha** en tablas de hechos → `event_date`.
- **Z-Ordering** (Delta Lake) sobre (`aircraft_id`, `event_ts`) para acelerar consultas por avión/ventana temporal.
- **Calidad de datos**: Validar rangos físicos (e.g., `egt_c`, `n1_pct`) y coherencia de fases de vuelo.
- **Features**: estadísticas móviles (5/15/60 min), deltas vs. baseline del avión, *lags* y *rolling std* sobre `vib_ips` y `egt_c`.


# 3.🎯 Objetivo del notebook

En este notebook vamos a **simular un flujo completo dentro de la arquitectura medallón**, centrado en las capas **Bronze → Silver → Quarantine**.  
El propósito es entender cómo estructurar un **Job de Databricks** que combine **procesos de validación, almacenamiento de errores y posterior recuperación de datos corregidos**.

---

## 🧩 Flujo general del Job

1. **Ingesta (Bronze)**  
   - Lectura de los datos originales desde una fuente cruda (por ejemplo, CSV o Parquet).  
   - Registro del esquema y almacenamiento sin transformaciones en la capa Bronze.

2. **Validación y aterrizaje (Silver)**  
   - Aplicación de reglas de calidad (tipos, duplicados, integridad referencial, valores nulos críticos, etc.).  
   - Inserción de los registros válidos en la tabla Silver.  
   - Derivación de los registros no válidos a la tabla de cuarentena.

3. **Cuarentena (Quarantine)**  
   - Almacenamiento de los registros rechazados junto con metadatos de error.  
   - Revisión y posible corrección manual o automatizada.

4. **Recuperación posterior**  
   - Reprocesamiento de los registros en cuarentena que fueron corregidos.  
   - Reinserción controlada en la tabla Silver tras validaciones adicionales.  

---

## 📊 Resultados esperados

- Tabla **Silver** con los datos válidos y consistentes.  
- Tabla **Quarantine** con los registros inicialmente rechazados.  
- Métricas que permitan evaluar la calidad de los datos:
  - Porcentaje de registros válidos vs en cuarentena.  
  - Evolución del número de registros corregidos y reincidentes.  

---


# 4. Creación de Repositorio en Unity Catalog

In [0]:
%sql
--drop catalog if exists sesion1_2 cascade;
drop schema if exists sesion1_2.layer20_bronze cascade;
drop schema if exists sesion1_2.layer30_silver cascade;
drop schema if exists sesion1_2.layer30_silver_quarantine cascade;

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS sesion1_2;

-- 01 · Landing / Raw
CREATE SCHEMA IF NOT EXISTS sesion1_2.layer10_landing;
CREATE VOLUME IF NOT EXISTS sesion1_2.layer10_landing.volume;

-- 02 · Bronze
CREATE SCHEMA IF NOT EXISTS sesion1_2.layer20_bronze;

-- 03 · Silver
CREATE SCHEMA IF NOT EXISTS sesion1_2.layer30_silver;
CREATE SCHEMA IF NOT EXISTS sesion1_2.layer30_silver_quarantine;

-- 04 · Gold
CREATE SCHEMA IF NOT EXISTS sesion1_2.layer40_gold;

-- 99 · Operaciones / Configuración
CREATE SCHEMA IF NOT EXISTS sesion1_2.layer99_ops;
CREATE VOLUME IF NOT EXISTS sesion1_2.layer99_ops.configs;


# 5. Ejercico

Ahora continuaremos con la creacion del JOB utilizando los .py preparados para ello

## 🛠️ 5. CREACIÓN DEL PRIMER WORKFLOW (Ingesta → Validación → Cuarentena)

---

### 📂 5.1 Ingesta — Preparación

- Coloca los CSV en el volumen:
`/Volumes/sesion1_2/layer10_landing/volume`

- Abre el archivo `01_ingesta_datos.py` y analiza:

> **❓ Preguntas**
> - ¿Qué argumentos son obligatorios?  
> - ¿El schema se impone o es flexible en esta fase? ¿Qué opinión te merece esta decisión?  
> - ¿Qué adiciones implementarías tú en esta fase de ingesta?  

---

### 🚀 5.2 Crea el primer **JOB de INGESTA** (tabla `dim_aircraft`)

**Parámetros recomendados:**

["--input_file=/Volumes/sesion1_2/layer10_landing/volume/fleet_dim_aircraft_errors3.csv", "--output_table=sesion1_2.layer20_bronze.dim_aircraft", "--write_mode=overwrite", "--partition_by="]


---

### ✅ 5.3 Validación de Datos

Abre el archivo `02_validacion_datos.py` y responde:

> **❓ Preguntas**
> - ¿Qué argumentos son obligatorios?  
> - Validaciones actuales incluidas:  
>   1. Nulos en Clave Primaria  
>   2. Duplicados en Clave Primaria  
>   3. Columnas numéricas fuera de rango  
>   4. Columnas no numéricas vacías  
>   5. Fecha bien formateada  
> - **¿Se te ocurre alguna validación adicional?**  
> - **¿Preferirías que en este punto fuera autoschema?**

---

### 🧪 5.4 Crea el **JOB de VALIDACIÓN** (tabla `dim_aircraft`)

**Parámetros recomendados:**

["--input_table=sesion1_2.layer20_bronze.dim_aircraft", "--output_table=sesion1_2.layer30_silver.dim_aircraft", "--config_path=/Volumes/sesion1_2/layer99_ops/configs/schemas_enriched.json", "--table_name=dim_aircraft", "--write_mode=overwrite", "--quarantine_table=sesion1_2.layer30_silver_quarantine.dim_aircraft_quarantine"]


---

### 🔁 5.5 Repite para las Tablas del Caso Real

Aplica los **dos JOBs anteriores** también sobre:

- `fact_engine_sensor`  
- `fact_maintenance_event`  

---

### 🚑 5.6 Notebook de **Recuperación de Cuarentena**

Abre el notebook `03_cuarentena_recov` y completa los ejercicios propuestos

> **❓ Reflexión**
> - ¿Qué opinas del uso de una tabla de cuarentena?  
> - ¿Crearías un **KPI de % de datos eliminados vs % recuperados** tras cuarentena?  
> - Completa las propuestas de lógica dentro del notebook.

---

### 🔗 5.7 Agrega el Notebook al Workflow

> **❓ Pregunta final**
> - ¿Cómo ves el workflow resultante?  
> - ¿Qué **siguiente dirección lógica** tomarías en un entorno real?
>   - Reporting / Dashboard  
>   - Automatizar alerts  
>   - Curación manual asistida  
>   - Limpieza iterativa Silver → Gold  

---

